[Advent of Code - Day 19](https://adventofcode.com/2023/day/19)

# Import *Workflow list and Part ratings*

In [1]:
import sys

sys.path.insert(0, "../")
from utils import aoc_input as inp

INPUT = inp.download_input(year="2023", day="19")

In [2]:
blank = INPUT.index("")

In [3]:
input_1 = [x.strip("}") for x in INPUT[:blank]]
input_2 = [x.strip("}").strip("{") for x in INPUT[blank + 1 :]]

In [4]:
input_1[:5], input_2[:5]

(['fmz{x<2152:R,s>3520:A,a<3391:A,A',
  'qhh{a>2045:A,s>3259:A,R',
  'xh{m<1065:bxz,spk',
  'kmp{m>2580:R,s>3306:A,x<420:R,R',
  'rdg{a<2587:md,tgj'],
 ['x=53,m=706,a=908,s=10',
  'x=518,m=1763,a=669,s=832',
  'x=745,m=1031,a=529,s=1094',
  'x=92,m=724,a=1146,s=1',
  'x=221,m=945,a=188,s=280'])

# *Accepted parts*

In [5]:
IN = "in"
ACCEPT, REJECT = "A", "R"

In [6]:
flows = {}

for line in input_1:
    flow, rules = line.split("{")
    rule = rules.split(",")
    flows[flow] = rule

parts = []

for line in input_2:
    parts.append(line.split(","))

## Part 1: Sum of *X-M-A-S ratings*

In [7]:
accept = []


def sorter(workflow, x, m, a, s):
    for step in workflow[:-1]:
        rule, dest = step.split(":")

        if eval(rule):
            return dest
    else:
        return workflow[-1]


for part in parts:
    current = IN
    for cat in part:
        exec(cat)

    while current not in [REJECT, ACCEPT]:
        current = sorter(flows[current], x, m, a, s)

    if current == ACCEPT:
        accept.append([x, m, a, s])

In [8]:
sum_1 = 0

for rating in accept:
    sum_1 += sum(rating)

sum_1

348378

## Part 2: Distinct *Rating combinations*

In [9]:
def update_ranges(cat, op, num, xl, xh, ml, mh, al, ah, sl, sh):
    if cat == "x":
        xl, xh = update_range(op, num, xl, xh)
    elif cat == "m":
        ml, mh = update_range(op, num, ml, mh)
    elif cat == "a":
        al, ah = update_range(op, num, al, ah)
    elif cat == "s":
        sl, sh = update_range(op, num, sl, sh)

    return (xl, xh, ml, mh, al, ah, sl, sh)


def update_range(op, num, lo, hi):
    if op == ">":
        lo = max(lo, num + 1)
    elif op == "<":
        hi = min(hi, num - 1)
    elif op == ">=":
        lo = max(lo, num)
    elif op == "<=":
        hi = min(hi, num)

    return (lo, hi)

In [10]:
queue = [(IN, (1, 4000, 1, 4000, 1, 4000, 1, 4000))]

res = 0

while queue:
    flow, ranges = queue.pop()
    xl, xh, ml, mh, al, ah, sl, sh = ranges
    if xl > xh or ml > mh or al > ah or sl > sh:
        continue

    if flow == REJECT:
        continue
    elif flow == ACCEPT:
        res += (xh - xl + 1) * (mh - ml + 1) * (ah - al + 1) * (sh - sl + 1)
        continue
    else:
        current_flow = flows[flow]

        for step in current_flow:
            dest = step
            if ":" in step:
                rule, dest = step.split(":")
                cat, op, num = rule[0], rule[1], int(rule[2:])
                queue.append((dest, update_ranges(cat, op, num, *ranges)))

                opp_op = ">=" if op == "<" else "<="
                ranges = update_ranges(cat, opp_op, num, *ranges)

            else:
                queue.append((dest, ranges))
                break

res

121158073425385